In [21]:
import sys
sys.path.append("../../src/earthtext")

from osm import osm
import os
from progressbar import progressbar as pbar
import shapely as sh 
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)
import geopandas as gpd
import pandas as pd
from importlib import reload
from rlxutils import Command
from time import time
import folium
reload(osm)

<module 'osm.osm' from '/home/ubuntu/earth-text/notebooks/osm/../../src/earthtext/osm/osm.py'>

## parts

In [3]:
part_ids = [i.split(".")[0] for i in os.listdir("/opt/data/osm/california-parts/") if i.endswith(".geojson")]

In [4]:
orig = []
for part_id in part_ids:
    with open(f"/opt/data/osm/california-parts/{part_id}.geojson") as f:
        geom = sh.from_geojson(f.read())
        orig.append([part_id, geom])

orig = gpd.GeoDataFrame(orig, columns=['part_id', 'geometry'], crs = epsg4326)

## chips

In [5]:
dg = gpd.read_file("/opt/data/california-worldcover-chips.fgb")
dg['chip_id'] = [osm.get_region_hash(i) for i in dg.geometry.values]

In [6]:
dg.head()

,col,row,geometry,chip_id
0,131584,119552,"POLYGON ((-114.01333 32.01600, -114.01333 32.0...",272274b1ba17e
1,131584,119296,"POLYGON ((-114.01333 32.03733, -114.01333 32.0...",257760bda8ca7
2,131328,119296,"POLYGON ((-114.03467 32.03733, -114.03467 32.0...",0630b39477419
3,131328,119552,"POLYGON ((-114.03467 32.01600, -114.03467 32.0...",166f6e23d38eb
4,131072,119552,"POLYGON ((-114.05600 32.01600, -114.05600 32.0...",0cb98e9e9788e


# check where are the chips with no osm data

In [92]:
dg_sample = dg[[not os.path.isfile(f"/opt/data/california-worldcover-chips/osm/{i}.parquet") for i in dg.chip_id]]
dg_sample.shape

(18557, 4)

In [ ]:
dg_sample.explore()

In [7]:
# get only chips with data
dgd = dg[[os.path.isfile(f"/opt/data/california-worldcover-chips/osm/{i}.parquet") for i in dg.chip_id]]
dgd.shape

(93363, 4)

In [8]:
dintersects_orig = []
orig_parts = []
for _,di in pbar(dgd.sample(10000).iterrows(), max_val = len(dgd)):
    #n_intersects = sum([oi.intersects(di.geometry) for oi in orig.geometry])
    intersects = orig[[oi.intersects(di.geometry) for oi in orig.geometry]]
    orig_parts.append(intersects)
    n_intersects = len(intersects)
    if n_intersects>1:
        dintersects_orig.append([di.chip_id, n_intersects])
    if n_intersects>2:
        break


/ |    #                                             | 27 Elapsed Time: 0:00:00

In [10]:
chip_ids = [i[0] for i in dintersects_orig if i[1]>1]

In [11]:
dgdi = dgd[dgd.chip_id.isin(chip_ids)]

In [18]:
frames = []
#for _, dg_sample in pbar(dgdi.iterrows(), max_val=len(dgdi)):
for _ in pbar(range(40)):
    _dg_sample = dgd.sample(1)
    dg_sample = _dg_sample.iloc[0]
    chip_id = dg_sample.chip_id
    z = gpd.read_parquet(f"/opt/data/california-worldcover-chips/osm/{chip_id}.parquet")
    if len(z)==0:
        print (f"skipping {chip_id}")
        continue
    z['tags'] = [osm.tags2str({k:v for k,v in t.items() if v is not None}) for t in z.tags]
    z['chip_id'] = chip_id
    
    g = dg_sample.geometry
    zi = z[[i.intersects(g) for i in z.geometry]]
    zi['geometry'] = [i.intersection(g) for i in zi.geometry]
    frames.append(zi)
    #frames.append(_dg_sample)

frames = pd.concat(frames)

  5% (2 of 40) |#                        | Elapsed Time: 0:00:00 ETA:   0:00:01

skipping 157a389436150


100% (40 of 40) |########################| Elapsed Time: 0:00:03 Time:  0:00:030000


skipping 0ca60aa0cfac9
skipping 2e0aaa15f32bd


In [ ]:
frames.explore(height=500)

In [ ]:
frames.explore(tiles='https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', 
               attr='esri',height = 500)
